# Text Extractor from PDF Files
## Provenance
* Created new 2025-05-17 at Bignor!

In [1]:
from pathlib import Path 
import pandas as pd
import re
import numpy as np
from functools import partial
from pprint import pprint as pp
from random import sample

import fitz  # PyMuPDF
import Levenshtein

import archivum.mendeley_port as arcm
import archivum.library as arcl
import archivum.utilities as arcu
import archivum.document as arcd
import archivum.crossref as arcc
import archivum.gui as arcg
import archivum.parser as arcp
import archivum.reference as arcr
import archivum.cli as cli 
from archivum.utilities import fGT

%load_ext autotime

time: 78 ms (started: 2025-05-30 22:43:48 +01:00)


In [2]:
lib = arcl.Library('uber-library')
lib.get_library_path_list()

[WindowsPath('C:/Users/steve/AppData/Local/archivum/books.archivum-config'),
 WindowsPath('C:/Users/steve/AppData/Local/archivum/uber-library.archivum-config')]

time: 16 ms (started: 2025-05-30 22:43:49 +01:00)


In [2]:
arcu.info(cli.entry)

id,<class 'click.core.Group'>
add_help_option,True
allow_extra_args,True
allow_interspersed_args,False
chain,False
commands,"{'open-library': <Command open-library>, 'save..."
context_settings,{}
deprecated,False
help,CLI for managing bibliographic entries.
hidden,False
ignore_unknown_options,False


time: 15 ms (started: 2025-05-30 22:11:32 +01:00)


In [7]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)

time: 109 ms (started: 2025-05-30 22:12:17 +01:00)


In [3]:
cli.entry(args=["list-libraries"], standalone_mode=False)

┍━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Library                      │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ books.archivum-config        │
├──────────────────────────────┤
│ uber-library.archivum-config │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙
time: 31 ms (started: 2025-05-30 22:43:55 +01:00)


In [4]:
cli.entry(args=["list-libraries", "-d"], standalone_mode=False)

┍━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ name         │ description                            │ bibtex_file                      │ pdf_dir_name   │ text_dir_name       │ extractor   │
┝━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ books        │ downloaded books                       │ \S\Telos\biblio\books-test.bib   │ \S\BOOKS       │ \temp\pdf-full-text │ pdftotext   │
├──────────────┼────────────────────────────────────────┼──────────────────────────────────┼────────────────┼─────────────────────┼─────────────┤
│ Uber Library │ Mendeley port from May 2025 plus books │ \S\Telos\biblio\uber-library.bib │ \S\Library     │ \temp\pdf-full-text │ pdftotext   │
┕━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━

In [5]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)

time: 31 ms (started: 2025-05-30 22:44:00 +01:00)


In [6]:
cli.entry(args=['get-library-stats'], standalone_mode=False)

┍━━━━━━━━━━━━━┯━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ index       │   references │   documents │
┝━━━━━━━━━━━━━┿━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ objects     │        6,118 │       6,132 │
├─────────────┼──────────────┼─────────────┤
│ no children │          167 │         156 │
├─────────────┼──────────────┼─────────────┤
│ children    │        5,951 │       5,976 │
├─────────────┼──────────────┼─────────────┤
│ 1 child     │        5,833 │       5,883 │
├─────────────┼──────────────┼─────────────┤
│ 2 children  │          108 │          86 │
├─────────────┼──────────────┼─────────────┤
│ 3 children  │           10 │           4 │
├─────────────┼──────────────┼─────────────┤
│ 4+ children │            0 │           3 │
┕━━━━━━━━━━━━━┷━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┙
time: 500 ms (started: 2025-05-30 22:44:01 +01:00)


In [21]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['get-distinct-values', ], standalone_mode=False)

┍━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━┑
│ index         │   distinct │   missing │
┝━━━━━━━━━━━━━━━┿━━━━━━━━━━━━┿━━━━━━━━━━━┥
│ tag           │      6,118 │         0 │
├───────────────┼────────────┼───────────┤
│ title         │      5,902 │         0 │
├───────────────┼────────────┼───────────┤
│ author        │      4,662 │         5 │
├───────────────┼────────────┼───────────┤
│ pages         │      2,749 │     2,233 │
├───────────────┼────────────┼───────────┤
│ doi           │      2,539 │     3,431 │
├───────────────┼────────────┼───────────┤
│ url           │      1,387 │     4,672 │
├───────────────┼────────────┼───────────┤
│ journal       │      1,225 │     2,643 │
├───────────────┼────────────┼───────────┤
│ volume        │        454 │     3,143 │
├───────────────┼────────────┼───────────┤
│ number        │        371 │     2,934 │
├───────────────┼────────────┼───────────┤
│ booktitle     │        241 │     5,842 │
├───────────────┼────────────┼───────────┤
│ mendeley-

In [22]:
cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['get-distinct-values', '-f author'], standalone_mode=False)

┍━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━┑
│ author                 │   count │
┝━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━┥
│ {Aon Benfield}         │      98 │
├────────────────────────┼─────────┤
│ Wang, Ruodu            │      78 │
├────────────────────────┼─────────┤
│ Mildenhall, Stephen J. │      56 │
├────────────────────────┼─────────┤
│ Delbaen, Freddy        │      49 │
├────────────────────────┼─────────┤
│ Marinacci, Massimo     │      48 │
├────────────────────────┼─────────┤
│ Embrechts, Paul        │      45 │
├────────────────────────┼─────────┤
│ Cummins, J. David      │      44 │
├────────────────────────┼─────────┤
│ Denuit, Michel M.      │      38 │
├────────────────────────┼─────────┤
│ {A. M. Best}           │      37 │
├────────────────────────┼─────────┤
│ Aon plc                │      36 │
├────────────────────────┼─────────┤
│ Bauer, Daniel          │      36 │
├────────────────────────┼─────────┤
│ Ghossoub, Mario        │      33 │
├────────────────────────┼─────────┤
│

In [23]:
cli.entry(args=['merge-library', 'othername'], standalone_mode=False)

22:15:10 | archivum.cli.merge_library     | INFO     | Merging othername into Library(uber-library.archivum-config)
22:15:10 | archivum.cli.merge_library     | TODO     | Implement merge_library!
time: 172 ms (started: 2025-05-30 22:15:10 +01:00)


In [24]:
cli.entry(args=['new'], standalone_mode=False)

22:15:12 | archivum.cli.new               | INFO     | Scanning directory C:\Users\steve\Downloads
┍━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   n │ document                                                     │
┝━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│   1 │ 1-s2.0-S016766872500037X-main.pdf                            │
├─────┼──────────────────────────────────────────────────────────────┤
│   2 │ 1-s2.0-S0167668725000599-main.pdf                            │
├─────┼──────────────────────────────────────────────────────────────┤
│   3 │ 1176996608.pdf                                               │
├─────┼──────────────────────────────────────────────────────────────┤
│   4 │ 12-AAP857.pdf                                                │
├─────┼──────────────────────────────────────────────────────────────┤
│   5 │ 2023-Dhaene-Robert-Cheung-Denuit.pdf                         │
├─────┼──────────────────────────────────────────

In [25]:
# cli.entry(args=['open-library', 'uber-library'], standalone_mode=False)
cli.entry(args=['close-library', ], standalone_mode=False)
cli.entry(args=['get-library-stats'], standalone_mode=False)

22:15:19 | archivum.cli.close_library     | INFO     | Closing library Library(uber-library.archivum-config)
22:15:19 | archivum.cli.close_library     | TODO     | SHOULD WE SAVE on CLOSE??
22:15:19 | archivum.library.close         | TODO     | Library.close()
time: 47 ms (started: 2025-05-30 22:15:19 +01:00)


In [82]:
cli.entry(args=["list-libraries", '-d'], standalone_mode=False)

112.0 {'name': 10.0, 'description': 32.0, 'bibtex_file': 32.0, 'pdf_dir_name': 10.0, 'text_dir_name': 19.0, 'extractor': 9.0}
┍━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ name    │ description                      │ bibtex_file                      │ pdf_dir_name   │ text_dir_name       │ extractor   │
┝━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ books   │ downloaded books                 │ \S\Telos\biblio\books-test.bib   │ \S\BOOKS       │ \temp\pdf-full-text │ pdftotext   │
├─────────┼──────────────────────────────────┼──────────────────────────────────┼────────────────┼─────────────────────┼─────────────┤
│ Uber    │ Mendeley port from May 2025 plus │ \S\Telos\biblio\uber-library.bib │ \S\Library     │ \temp\pdf-full-text │ pdftotext   │
│ Library │ books                            │                  

In [26]:
result = lib.database.querex('top 5 recent verbose')
print(fGT(result))

┍━━━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━┯━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┑
│    mod     │ tag         │ type    │   year │ author          │ title                                         │   create   │ tpath         │
┝━━━━━━━━━━━━┿━━━━━━━━━━━━━┿━━━━━━━━━┿━━━━━━━━┿━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━┿━━━━━━━━━━━━━━━┥
│ 2025-05-13 │ Schmidt2003 │ article │   2003 │ Schmidt,        │ The true invariant of an arbitrage free       │ 2025-05-13 │ Schmidt       │
│            │             │         │        │ Anatoly B.      │ portfolio                                     │            │               │
├────────────┼─────────────┼─────────┼────────┼─────────────────┼───────────────────────────────────────────────┼────────────┼───────────────┤
│ 2025-05-12 │ Tanner1987  │ article │   1987 │ Tanner, Martin  │ The Calculation of Posterior Distributions by │ 2025-05-12 │ Tanner, Wong  │

In [29]:
f = fGT(lib.querex('top 10 recent where year == 2024 !/Wang, R/ sort title'))
print(f)

┍━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━┯━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┑
│    mod     │ tag           │ type    │   year │ author   │ title                                                 │   create   │ tpath           │
┝━━━━━━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━━━┿━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━┥
│ 2025-01-02 │ Liu2024       │ article │   2024 │ Wang,    │ Quantiles under ambiguity and risk sharing            │ 2025-01-02 │ Liu, Mao, Wang  │
│            │               │         │        │ Ruodu    │                                                       │            │                 │
├────────────┼───────────────┼─────────┼────────┼──────────┼───────────────────────────────────────────────────────┼────────────┼─────────────────┤
│ 2024-12-06 │ Fadina2024    │ article │   2024 │ Wang,    │ A framework for measures of risk under uncertainty 

In [30]:
f._debug_col_widths

,break_penalties,breakable_cols,min_possible_widths,ideal_widths,col_widths,headers_cw,headers_ideal,headers_min
mod,5.0,False,10.0,10.0,10.0,3.0,3.0,3.0
tag,1.0,True,13.0,13.0,13.0,3.0,3.0,3.0
type,1.0,True,7.0,7.0,7.0,4.0,4.0,4.0
year,1.0,True,4.0,4.0,4.0,4.0,4.0,4.0
author,1.0,True,5.0,11.0,8.0,6.0,6.0,6.0
title,1.0,True,16.0,104.0,53.0,5.0,5.0,5.0
create,5.0,False,10.0,10.0,10.0,6.0,6.0,6.0
tpath,1.0,True,9.0,24.0,15.0,5.0,5.0,5.0
total,16.0,6.0,74.0,183.0,120.0,36.0,36.0,36.0


time: 15 ms (started: 2025-05-30 22:18:33 +01:00)


In [31]:
f = fGT(lib.database.querex('top 10 recent !/Wang, R/'))
print(f)

┍━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━┯━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┑
│    mod     │ tag            │ type        │   year │ author   │ title                                         │   create   │ tpath            │
┝━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┿━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━┥
│ 2025-03-03 │ Cote2025       │ article     │   2025 │ Wang,    │ On convex order and supermodular order        │ 2025-03-03 │ Côté, Wang       │
│            │                │             │        │ Ruodu    │ without finite mean                           │            │                  │
├────────────┼────────────────┼─────────────┼────────┼──────────┼───────────────────────────────────────────────┼────────────┼──────────────────┤
│ 2025-03-03 │ Chen2025a      │ unpublished │   2025 │ Wang,    │ Diversification for infinite-mean Pareto      │ 2025-03-03

# New files

In [34]:
cli.LibraryContext.last_new

,n,document,path
0,1,1-s2.0-S016766872500037X-main.pdf,C:\Users\steve\Downloads\1-s2.0-S0167668725000...
1,2,1-s2.0-S0167668725000599-main.pdf,C:\Users\steve\Downloads\1-s2.0-S0167668725000...
2,3,1176996608.pdf,C:\Users\steve\Downloads\1176996608.pdf
3,4,12-AAP857.pdf,C:\Users\steve\Downloads\12-AAP857.pdf
4,5,2023-Dhaene-Robert-Cheung-Denuit.pdf,C:\Users\steve\Downloads\2023-Dhaene-Robert-Ch...
5,6,2305.17743v2.pdf,C:\Users\steve\Downloads\2305.17743v2.pdf
6,7,304893.304937.pdf,C:\Users\steve\Downloads\304893.304937.pdf
7,8,3_Obstfeld.pdf,C:\Users\steve\Downloads\3_Obstfeld.pdf
8,9,9309211v2.pdf,C:\Users\steve\Downloads\9309211v2.pdf
9,10,AlphaEvolve.pdf,C:\Users\steve\Downloads\AlphaEvolve.pdf


time: 15 ms (started: 2025-05-30 23:34:46 +01:00)


In [39]:
cli.entry(args=['new', '-m'], standalone_mode=False)

23:41:22 | archivum.cli.new               | INFO     | Scanning directory C:\Users\steve\Downloads
23:41:22 | archivum.cli.new               | INFO     | Displaying metadata for found PDFs.
┍━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   n │   create   │ document                           │ meta_author      │ meta_subject                  │ meta_title                   │
┝━━━━━┿━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│  24 │ 2025-05-30 │ Towards_a_theory_of_tables.pdf     │                  │                               │                              │
├─────┼────────────┼────────────────────────────────────┼──────────────────┼───────────────────────────────┼──────────────────────────────┤
│  23 │ 2025-05-30 │ Toward_tighter_tables.pdf          │                  │                               │  

In [29]:
df = cli.LibraryContext.last_new
df

,n,document,path
0,1,1-s2.0-S016766872500037X-main.pdf,C:\Users\steve\Downloads\1-s2.0-S0167668725000...
1,2,1-s2.0-S0167668725000599-main.pdf,C:\Users\steve\Downloads\1-s2.0-S0167668725000...
2,3,1176996608.pdf,C:\Users\steve\Downloads\1176996608.pdf
3,4,12-AAP857.pdf,C:\Users\steve\Downloads\12-AAP857.pdf
4,5,2023-Dhaene-Robert-Cheung-Denuit.pdf,C:\Users\steve\Downloads\2023-Dhaene-Robert-Ch...
5,6,2305.17743v2.pdf,C:\Users\steve\Downloads\2305.17743v2.pdf
6,7,304893.304937.pdf,C:\Users\steve\Downloads\304893.304937.pdf
7,8,3_Obstfeld.pdf,C:\Users\steve\Downloads\3_Obstfeld.pdf
8,9,9309211v2.pdf,C:\Users\steve\Downloads\9309211v2.pdf
9,10,AlphaEvolve.pdf,C:\Users\steve\Downloads\AlphaEvolve.pdf


time: 15 ms (started: 2025-05-30 23:32:16 +01:00)


In [11]:
pdfs = list((Path().home() / 'Downloads').glob('*.pdf'))
pdfs

[WindowsPath('C:/Users/steve/Downloads/1-s2.0-S016766872500037X-main.pdf'),
 WindowsPath('C:/Users/steve/Downloads/1-s2.0-S0167668725000599-main.pdf'),
 WindowsPath('C:/Users/steve/Downloads/1176996608.pdf'),
 WindowsPath('C:/Users/steve/Downloads/12-AAP857.pdf'),
 WindowsPath('C:/Users/steve/Downloads/2023-Dhaene-Robert-Cheung-Denuit.pdf'),
 WindowsPath('C:/Users/steve/Downloads/2305.17743v2.pdf'),
 WindowsPath('C:/Users/steve/Downloads/304893.304937.pdf'),
 WindowsPath('C:/Users/steve/Downloads/3_Obstfeld.pdf'),
 WindowsPath('C:/Users/steve/Downloads/9309211v2.pdf'),
 WindowsPath('C:/Users/steve/Downloads/AlphaEvolve.pdf'),
 WindowsPath('C:/Users/steve/Downloads/CT 12 10738 - Legacy Filing Packet 08-14-2012 {AFM ap}.pdf'),
 WindowsPath('C:/Users/steve/Downloads/Explanatory Memorandum.pdf'),
 WindowsPath('C:/Users/steve/Downloads/Gaussian emergence - accompanying note.pdf'),
 WindowsPath('C:/Users/steve/Downloads/Gaussian emergence for stochastic reserve risk modelling.pdf'),
 Windows

time: 0 ns (started: 2025-05-30 22:47:55 +01:00)


In [14]:
for p in pdfs[:1]:
    d = arcd.Document(p)
d

Document(1-s2.0-S016766872500037X-main.pdf)

time: 16 ms (started: 2025-05-30 22:49:20 +01:00)


In [16]:
d.stats

{'name': '1-s2.0-S016766872500037X-main.pdf',
 'path': 'C:/Users/steve/Downloads/1-s2.0-S016766872500037X-main.pdf',
 'mod': Timestamp('2025-03-07 08:38:18.099158+0000', tz='Europe/London'),
 'create': Timestamp('2025-03-07 08:38:17.517345400+0000', tz='Europe/London'),
 'access': Timestamp('2025-05-28 10:20:12.449331100+0100', tz='Europe/London'),
 'node': 5629499535417802,
 'links': 1,
 'size': 1713793,
 'suffix': 'pdf'}

time: 0 ns (started: 2025-05-30 22:49:38 +01:00)


In [ ]:
md = d.meta_data(lib=lib)

In [21]:
md.author, md.subject, md.title, md.author_ex

('Christopher Blier-Wong',
 'Insurance Mathematics and Economics, 122 (2025) 119-136. doi:10.1016/j.insmatheco.2025.02.006',
 'Efficient evaluation of risk allocations',
 'Blier-Wong, Christopher')

time: 0 ns (started: 2025-05-30 22:54:06 +01:00)


In [24]:
import pycomfy

time: 0 ns (started: 2025-05-30 23:31:26 +01:00)


In [33]:
pycomfy.render_table?

time: 0 ns (started: 2025-05-30 23:33:19 +01:00)


Signature: pycomfy.render_table(rows)
Docstring: <no docstring>
Type:      builtin_function_or_method

In [25]:
rows = [
    ["Name", "Role", "Location"],
    ["Steve", "Actuary", "London"],
    ["Mildy", "Risk Analyst", "UK"],
]

print(pycomfy.render_table(rows))

┌───────┬──────────────┬──────────┐
│ Name  ┆ Role         ┆ Location │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ Steve ┆ Actuary      ┆ London   │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ Mildy ┆ Risk Analyst ┆ UK       │
└───────┴──────────────┴──────────┘
time: 0 ns (started: 2025-05-30 23:31:28 +01:00)


In [32]:
print(pycomfy.render_table(df.astype(str).values))

┌────┬──────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────┐
│ 1  ┆ 1-s2.0-S016766872500037X-main.pdf                            ┆ C:\Users\steve\Downloads\1-s2.0-S016766872500037X-main.pdf                            │
├╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2  ┆ 1-s2.0-S0167668725000599-main.pdf                            ┆ C:\Users\steve\Downloads\1-s2.0-S0167668725000599-main.pdf                            │
├╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3  ┆ 1176996608.pdf                                               ┆ C:\Users\steve\Downloads\1176996608.pdf                                               │
├╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [21]:
arcp.parser('top 10 recent title ~ /Risk Measure/')

{'select': {'include': [], 'exclude': []},
 'sort': [],
 'regex': [('title', 'Risk Measure')],
 'where': None,
 'top': 10,
 'flags': ['recent']}

time: 31 ms (started: 2025-05-29 23:58:36 +01:00)


In [31]:
lib.ref_df.columns

Index(['tag', 'type', 'author', 'doi', 'journal', 'pages', 'title', 'volume',
       'year', 'publisher', 'url', 'institution', 'number', 'mendeley-tags',
       'booktitle', 'edition', 'month', 'address', 'editor', 'arc-citations',
       'arc-source'],
      dtype='object')

time: 0 ns (started: 2025-05-30 00:18:11 +01:00)


In [33]:
x = lib.ref_df[['tag', 'type', 'author', 'title']].head().to_string(index=False)
print(x)

             tag       type                                                  author                                                                                        title
     Schmidt2003    article                                     Schmidt, Anatoly B.                                          {The true invariant of an arbitrage free portfolio}
   Bahnemann2015       book                                        Bahnemann, David                                                                {Distributions for Actuaries}
Khoshnevisan2011 techreport                                     Khoshnevisan, Davar                                                      {Topics in Probability: Levy Processes}
       Mahul2003    article                                          Mahul, Olivier                                 {Efficient Risk Sharing within a Catastrophe Insurance Pool}
       Jiang2025    article Jiang, By John X. and Koo, David S. and Wang, Isabel Y. {The Road Not Taken: Evaluating

In [26]:
lib.ref_df.to_csv('ref_df.csv', encoding='utf-8')

time: 47 ms (started: 2025-05-30 00:14:00 +01:00)


In [25]:
fGT(lib.ref_df.querex('top 10 recent title ~ /Collapse to the Mean/'))

tag,type,year,author,title
Centrone2025,article,2025,"Centrone, Francesca and {Rosazza Gianin}, Emanuela",Capital Allocation Rules and Generalized Collapse to the Mean
Liebrich2022,article,2022,"Liebrich, Felix Benedikt and Munari, Cosimo",Law-invariant functionals that collapse to the mean: beyond convexity
Bellini2021,article,2021,"Bellini, Fabio and Koch-Medina, Pablo and Munari, Cosimo and Svindland, Gregor",Law-invariant functionals that collapse to the mean
Bellini2021a,article,2021,"Bellini, Fabio and Koch-Medina, Pablo and Munari, Cosimo and Svindland, Gregor",Law-invariant functionals that collapse to the mean
Chen2021a,article,2021,"Chen, Shengzhong and Gao, Niushan and Leung, Denny H. and Li, Lei",Do law-invariant linear functionals collapse to the mean?


time: 62 ms (started: 2025-05-30 00:00:47 +01:00)


In [5]:
lib.stats_ref_fields()

,distinct,missing
tag,6118,0
type,10,0
author,4662,5
doi,2539,3431
journal,1225,2643
pages,2749,2233
title,5902,0
volume,454,3143
year,100,256
publisher,169,5579


time: 47 ms (started: 2025-05-28 14:29:06 +01:00)


In [186]:
ps2 = list(Path('\\users\\steve\\downloads').glob('*.pdf'))
ps2

[WindowsPath('/users/steve/downloads/1-s2.0-S016766872500037X-main.pdf'),
 WindowsPath('/users/steve/downloads/1-s2.0-S0167668725000599-main.pdf'),
 WindowsPath('/users/steve/downloads/1176996608.pdf'),
 WindowsPath('/users/steve/downloads/12-AAP857.pdf'),
 WindowsPath('/users/steve/downloads/2023-Dhaene-Robert-Cheung-Denuit.pdf'),
 WindowsPath('/users/steve/downloads/2305.17743v2.pdf'),
 WindowsPath('/users/steve/downloads/3_Obstfeld.pdf'),
 WindowsPath('/users/steve/downloads/9309211v2.pdf'),
 WindowsPath('/users/steve/downloads/CT 12 10738 - Legacy Filing Packet 08-14-2012 {AFM ap}.pdf'),
 WindowsPath('/users/steve/downloads/Explanatory Memorandum.pdf'),
 WindowsPath('/users/steve/downloads/Gaussian emergence - accompanying note.pdf'),
 WindowsPath('/users/steve/downloads/Gaussian emergence for stochastic reserve risk modelling.pdf'),
 WindowsPath('/users/steve/downloads/gross-zagier.pdf'),
 WindowsPath('/users/steve/downloads/NJ HO 2024 Filing.pdf'),
 WindowsPath('/users/steve/down

time: 0 ns (started: 2025-05-28 10:18:09 +01:00)


In [ ]:
text_dir_name='\\temp\\pdf-full-text'
extractor='pdftotext'
pDocument = partial(arcd.Document, text_dir_path=Path(text_dir_name), extractor=extractor)
ps = lib.doc_df.sample(200).path

ans = []
for p in ps:
    p = Path(p)
    d = pDocument(p)
    a = d.meta_data(lib)
    ans.append([p.name, a.author, a.author_ex, a.subject, a.title, a.raw])

df = pd.DataFrame(ans, columns=['name', 'author', 'author_ex', 'subject', 'title', 'raw'])
fGT(df)

# GUI

In [10]:
def edit_book_info():
    import tkinter as tk
    from tkinter import messagebox

    def on_done():
        nonlocal result
        result = {
            "Title": title_var.get(),
            "Author": author_var.get(),
            "Year": year_var.get()
        }
        root.destroy()

    def on_cancel():
        root.destroy()

    result = None
    root = tk.Tk()
    root.title("Edit Book")

    tk.Label(root, text="Title").grid(row=0, column=0, sticky="e")
    tk.Label(root, text="Author").grid(row=1, column=0, sticky="e")
    tk.Label(root, text="Year").grid(row=2, column=0, sticky="e")

    title_var = tk.StringVar(value="The Great Gatsby")
    author_var = tk.StringVar(value="F. Scott Fitzgerald")
    year_var = tk.StringVar(value="1925")

    tk.Entry(root, textvariable=title_var).grid(row=0, column=1)
    tk.Entry(root, textvariable=author_var).grid(row=1, column=1)
    tk.Entry(root, textvariable=year_var).grid(row=2, column=1)

    tk.Button(root, text="Cancel", command=on_cancel).grid(row=3, column=0, pady=10)
    tk.Button(root, text="Done", command=on_done).grid(row=3, column=1, pady=10)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-27 23:20:03 +01:00)


In [12]:
import tkinter as tk

def edit_fields(data, mandatory=(), title="Edit Info"):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    root = tk.Tk()
    root.title(title)

    vars_ = {}
    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=2)
        var = tk.StringVar(value=value)
        vars_[key] = var
        tk.Entry(root, textvariable=var).grid(row=row, column=1, padx=5, pady=2)

    row += 1
    tk.Button(root, text="Cancel", width=10, command=on_cancel).grid(row=row, column=0, pady=10)
    tk.Button(root, text="Done", width=10, command=on_done).grid(row=row, column=1, pady=10)

    root.mainloop()
    return result


time: 16 ms (started: 2025-05-27 23:23:06 +01:00)


In [13]:
bi = {'title': "Once upon a time in America", 
     'author': 'LeLilo, Don.', 
     'year': 2008}
mand = ['title', 'publisher']

time: 0 ns (started: 2025-05-27 23:23:51 +01:00)


In [14]:
edit_fields(bi, mand)

{'title': 'Once upon a time in America',
 'author': 'LeLilo, Don.',
 'year': '2008',
 'publisher': ''}

time: 13.8 s (started: 2025-05-27 23:24:01 +01:00)


In [11]:
edit_book_info()

{'Title': 'The Great Gatsby', 'Author': 'F. Scott Fitzgerald', 'Year': '1925'}

time: 20.4 s (started: 2025-05-27 23:20:11 +01:00)


In [48]:
import tkinter as tk
from tkinter import ttk

def edit_fields(data, mandatory=(), title="Edit Info", known_values=None):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")

    known_values = known_values or {}
    vars_ = {}

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    def setup_combobox(box, values):
        def on_key(event):
            val = box.get().lower()
            matches = [v for v in values if val in v.lower()]
            if matches:
                box['values'] = matches
            else:
                box['values'] = values
        box.bind('<KeyRelease>', on_key)
        
    root = tk.Tk()
    root.title(title)
    root.geometry("700x400")  # wider window

    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar(value=value)
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key])
            box.config(width=80)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=80).grid(row=row, column=1, padx=5, pady=4)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=10)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="right", padx=10)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-27 23:39:52 +01:00)


In [71]:
import tkinter as tk
from tkinter import ttk
from rapidfuzz import process

def edit_fields(data, mandatory=(), title="Edit Info", known_values=None):
    result = None
    fields = dict(data)
    for key in mandatory:
        fields.setdefault(key, "")
    
    FONT = ("Segoe UI", 10)

    known_values = known_values or {}
    vars_ = {}

    def on_done():
        nonlocal result
        result = {k: vars_[k].get() for k in fields}
        root.destroy()

    def on_cancel():
        root.destroy()

    def on_next():

    def on_previous()
    
    def setup_fuzzy_combobox(box, values):
        def on_key(event):
            current = box.get()
            if current:
                match, score, _ = process.extractOne(current, values, score_cutoff=40)
                if match:
                    box.set(match)
        box.bind('<FocusOut>', on_key)
        box.bind('<Tab>', on_key)

    root = tk.Tk()
    root.title(title)
    root.geometry("800x500")

    for row, (key, value) in enumerate(fields.items()):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar(value=value)
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key], width=80, font=FONT)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_fuzzy_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=82, font=FONT).grid(row=row, column=1, padx=5, pady=4)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=10)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="right", padx=10)
    tk.Button(btn_frame, text="Previous", width=12, command=on_previous).pack(side="right", padx=10)
    tk.Button(btn_frame, text="Next", width=12, command=on_next).pack(side="right", padx=10)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-27 23:54:36 +01:00)


In [76]:
import tkinter as tk
from tkinter import ttk
from rapidfuzz import process
import pandas as pd

def edit_fields_from_df(df, i, mandatory=(), title="Edit Row", known_values=None):
    result = None
    rowcount = len(df)
    fields = df.iloc[i].to_dict()
    for key in mandatory:
        fields.setdefault(key, "")

    known_values = known_values or {}
    vars_ = {}

    def load_row(idx):
        for k in fields:
            val = str(df.iloc[idx].get(k, ""))
            vars_[k].set(val)

    def save_row():
        return {k: vars_[k].get() for k in fields}

    def on_done():
        nonlocal result
        result = (save_row(), current[0])
        root.destroy()

    def on_cancel():
        root.destroy()

    def on_prev():
        if current[0] > 0:
            current[0] -= 1
            load_row(current[0])

    def on_next():
        if current[0] < rowcount - 1:
            current[0] += 1
            load_row(current[0])

    def setup_fuzzy_combobox(box, values):
        def on_key(event):
            current = box.get()
            if current:
                match, score, _ = process.extractOne(current, values, score_cutoff=70)
                if match and match != current:
                    box.set(match)
                    box.icursor(tk.END)
        box.bind('<KeyRelease>', on_key)

    current = [i]  # mutable index holder
    root = tk.Tk()
    root.title(title)
    root.geometry("800x500")

    for row, key in enumerate(fields):
        tk.Label(root, text=key).grid(row=row, column=0, sticky="e", padx=5, pady=4)
        var = tk.StringVar()
        vars_[key] = var

        if key in known_values:
            box = ttk.Combobox(root, textvariable=var, values=known_values[key], width=80)
            box.grid(row=row, column=1, padx=5, pady=4)
            setup_fuzzy_combobox(box, known_values[key])
        else:
            tk.Entry(root, textvariable=var, width=80).grid(row=row, column=1, padx=5, pady=4)

    load_row(i)

    row += 1
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=row, column=0, columnspan=2, pady=20)
    tk.Button(btn_frame, text="Previous", width=12, command=on_prev).pack(side="left", padx=5)
    tk.Button(btn_frame, text="Next", width=12, command=on_next).pack(side="left", padx=5)
    tk.Button(btn_frame, text="Cancel", width=12, command=on_cancel).pack(side="left", padx=20)
    tk.Button(btn_frame, text="Done", width=12, command=on_done).pack(side="left", padx=5)

    root.mainloop()
    return result


time: 0 ns (started: 2025-05-28 00:02:34 +01:00)


In [80]:
mandatory = ['type', 'author', 'title', 'year', 'journal', 'publisher']
defaults = {
    k: lib.distinct(k) for k in mandatory
}

time: 32 ms (started: 2025-05-28 00:04:24 +01:00)


In [81]:
# def edit_fields_from_df(df, i, mandatory=(), title="Edit Row", known_values=None):
updated = edit_fields_from_df(lib.ref_df.sample(20).reset_index(drop=True)[mandatory], 0, mandatory, title="Edit Book Info", known_values=defaults)
updated

time: 58.5 s (started: 2025-05-28 00:04:29 +01:00)


In [79]:
lib.ref_df.sample(20).reset_index(drop=True).head().T#[mandatory]

,0,1,2,3,4
tag,Peng2019,Cummins2009a,Ali2017,Keich2005,Ridley1976
type,article,article,article,article,book
author,"Peng, Shige","Cummins, J. David and Weiss, Mary A. and Zi, Hongmin","Ali, Muneeb and Shea, Ryan and Nelson, Jude and Freedman, Michael J.","Keich, Uri","Ridley, B. K."
doi,10.1186/s41546-019-0038-2,10.1287/mnsc.45.9.1254,,,10.1119/1.11114
journal,"Probability, Uncertainty and Quantitative Risk",Management Science,,Journal of Computational Biology,
pages,1--8,1254--1269,,416--430,
title,{Law of large numbers and central limit theorem under nonlinear expectations},{Organizational Form and Efficiency: The Coexistence of Stock and Mutual Property-Liability Insu...,{Blockstack: A New Internet for Decentralized Applications},{sFFT: A Faster Accurate Computation of the p-Value of the Entropy Score},"{Time, Space and Things}"
volume,4,45,,12,
year,2019,2009,2017,2005,1976
publisher,,,,,Penguin Books Ltd.


time: 0 ns (started: 2025-05-28 00:04:09 +01:00)


In [72]:
i = i + 1
data = {k:v for k, v in lib.ref_df.loc[i].items() if v != ''}
updated = edit_fields(data, mandatory, title="Edit Book Info", known_values=defaults)
updated

time: 4.7 s (started: 2025-05-27 23:54:36 +01:00)


In [19]:
lib = arcl.Library('uber-library')

time: 31 ms (started: 2025-05-27 23:29:58 +01:00)


In [63]:
i = 202

time: 0 ns (started: 2025-05-27 23:49:21 +01:00)


time: 0 ns (started: 2025-05-27 23:34:26 +01:00)


time: 16 ms (started: 2025-05-27 23:34:28 +01:00)


In [40]:
data = lib.ref_df.querex('select journal, publisher tag ~ Mildenhall2024')[mandatory].iloc[0].to_dict()

time: 15 ms (started: 2025-05-27 23:34:48 +01:00)


{'tag': 'Shen2024',
 'type': 'article',
 'author': 'Shen, Yi and {Van Oosten}, Zachary and Wang, Ruodu',
 'pages': '1--51',
 'title': '{Partially Law-Invariant Risk Measures}',
 'year': '2024',
 'url': 'http://arxiv.org/abs/2401.17265',
 'number': 'June',
 'mendeley-tags': 'WangR',
 'arc-citations': 0,
 'arc-source': 'mendeley'}

time: 0 ns (started: 2025-05-27 23:43:02 +01:00)


In [18]:

data = {"Title": "1984", "Author": "George Orwell", 'Genre': 'Fiction|'}
mandatory = ["Title", "Author", "Year", "Genre"]
known = {"Genre": ["Fiction", "Non-fiction", "Sci-Fi", "Fantasy", "Other"]}
updated = edit_fields(data, mandatory, title="Edit Book Info", known_values=known)
print(updated)


{'Title': '1984', 'Author': 'George Orwell', 'Genre': 'Fiction|', 'Year': ''}
time: 16.3 s (started: 2025-05-27 23:29:16 +01:00)


In [51]:
defaults['type']

['Report',
 'article',
 'book',
 'incollection',
 'inproceedings',
 'manual',
 'misc',
 'phdthesis',
 'techreport',
 'unpublished']

time: 16 ms (started: 2025-05-27 23:41:28 +01:00)


In [200]:
lib._trie = None

time: 0 ns (started: 2025-05-28 10:55:25 +01:00)


In [203]:
lib.to_name_ex('Tsanakas, Andr')

Tsanakas, Andr is not a key...adding


'Tsanakas, Andreas'

time: 15 ms (started: 2025-05-28 10:55:55 +01:00)


In [82]:
import requests

def lookup_doi(doi: str) -> dict | None:
    url = f"https://api.crossref.org/works/{doi}"
    resp = requests.get(url, headers={"User-Agent": "DOI lookup script"})
    if resp.status_code == 200:
        return resp.json()["message"]
    return None


time: 156 ms (started: 2025-05-28 00:06:29 +01:00)


In [83]:
data = lookup_doi("10.1038/nphys1170")
if data:
    print(data["title"][0])
    print(data["author"][0]["family"], data["author"][0]["given"])


Measured measurement
Aspelmeyer Markus
time: 1.11 s (started: 2025-05-28 00:06:42 +01:00)


In [91]:
def extract_bibtex_fields(data: dict) -> dict:
    authors = data.get("author", [])
    author_str = " and ".join(
        f"{a['family']}, {a.get('given', '')}".strip() for a in authors
    )

    return {
        "title": data.get("title", [""])[0],
        "author": author_str,
        "year": data.get("issued", {}).get("date-parts", [[None]])[0][0],
        "journal": data.get("container-title", [""])[0],
        "volume": data.get("volume", ""),
        "number": data.get("issue", ""),
        "pages": data.get("page", ""),
        "doi": data.get("DOI", ""),
        "url": data.get("URL", "")
    }


time: 16 ms (started: 2025-05-28 00:10:55 +01:00)


In [ ]:
for d in dois[40:45]:
    data = lookup_doi(d)
    if data:
        print(d)
        print('-' * len(d))
        print(extract_bibtex_fields(data))
        print()
    else:
        print(f'FAIL {d}\n')

In [95]:
len(dois)

2538

time: 0 ns (started: 2025-05-28 00:12:29 +01:00)


In [84]:
dois = lib.distinct('doi')

time: 0 ns (started: 2025-05-28 00:07:03 +01:00)


In [96]:
def search_doi_by_title(title: str) -> dict | None:
    url = "https://api.crossref.org/works"
    params = {"query.title": title, "rows": 1}
    headers = {
        "User-Agent": "archivum/1.1 (mailto:mynl@me.com)"
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.status_code == 200:
        items = resp.json()["message"]["items"]
        return items[0] if items else None
    return None


time: 0 ns (started: 2025-05-28 00:13:10 +01:00)


In [102]:
result = search_doi_by_title()
if result:
    print(result["DOI"])

10.1007/bf01388809
time: 2.75 s (started: 2025-05-28 00:19:46 +01:00)


In [103]:
extract_bibtex_fields(result)

{'title': 'Heegner points and derivatives ofL-series',
 'author': 'Gross, Benedict H. and Zagier, Don B.',
 'year': 1986,
 'journal': 'Inventiones Mathematicae',
 'volume': '84',
 'number': '2',
 'pages': '225-320',
 'doi': '10.1007/bf01388809',
 'url': 'https://doi.org/10.1007/bf01388809'}

time: 16 ms (started: 2025-05-28 00:19:51 +01:00)


In [100]:
from pprint import pprint

time: 0 ns (started: 2025-05-28 00:18:12 +01:00)


In [ ]:
pprint(result)

In [220]:
lib

Library(uber-library.archivum-config)

time: 0 ns (started: 2025-05-28 11:44:31 +01:00)


In [223]:
len(r)

1

time: 0 ns (started: 2025-05-28 11:45:44 +01:00)


In [ ]:
r = arcc.search(title="Heegner points and derivatives of L-series", rows=1)
arcc.to_bibtex(r[0])

arcc.to_bibtex(r[0], lib)

In [226]:
arcc.to_bibtex(r[0], lib)

{'type': 'article',
 'title': 'Heegner points and derivatives ofL-series',
 'year': 1986,
 'doi': '10.1007/bf01388809',
 'url': 'https://doi.org/10.1007/bf01388809',
 'author': 'Gross, Benedict H. and Zagier, Don B.',
 'journal': 'Inventiones Mathematicae',
 'volume': '84',
 'number': '2',
 'pages': '225-320'}

time: 0 ns (started: 2025-05-28 11:47:01 +01:00)


In [231]:
r = arcr.Reference('Mild2025', 'journal', 'My splendid thing', 2025, author='Mildenhall, Stephen J.')

time: 0 ns (started: 2025-05-28 13:50:42 +01:00)


In [232]:
r

Reference(tag='Mild2025', type='journal', title='My splendid thing', year=2025, author='Mildenhall, Stephen J.', journal=None, volume=None, number=None, month=None, pages=None, edition=None, booktitle=None, editor=None, publisher=None, institution=None, address=None, doi=None, url=None, file=None, source=None, mendeley_tags=None, arc_citations=None, arc_source=None)

time: 0 ns (started: 2025-05-28 13:50:48 +01:00)


In [239]:
r.to_dict(fillna='')

{'tag': 'Mild2025',
 'type': 'journal',
 'title': 'My splendid thing',
 'year': 2025,
 'author': 'Mildenhall, Stephen J.',
 'journal': '',
 'volume': '',
 'number': '',
 'month': '',
 'pages': '',
 'edition': '',
 'booktitle': '',
 'editor': '',
 'publisher': '',
 'institution': '',
 'address': '',
 'doi': '',
 'url': '',
 'file': '',
 'source': '',
 'mendeley_tags': '',
 'arc_citations': '',
 'arc_source': ''}

time: 0 ns (started: 2025-05-28 13:58:26 +01:00)


In [ ]:
r = arcc.search(author='Stephen Mildenhall', rows=5)
print(len(r))

for _ in r:
    ref = arcr.Reference.from_crossref(_, lib)
    pp(ref.to_dict())

In [47]:
lib.distinct('arc-source')

['mendeley']

time: 16 ms (started: 2025-05-28 15:03:19 +01:00)


In [43]:
pp(ref.to_dict())

{'author': 'Mildenhall, Stephen J.',
 'doi': '10.3390/risks5020031',
 'journal': 'Risks',
 'number': '2',
 'pages': '31',
 'source': 'crossref',
 'tag': 'Mildenhall2017',
 'title': 'Actuarial Geometry',
 'type': 'article',
 'url': 'https://doi.org/10.3390/risks5020031',
 'volume': '5',
 'year': 2017}
time: 0 ns (started: 2025-05-28 15:01:57 +01:00)


In [67]:
s = [i for i in lib.distinct('tag') if re.search(r'^AMBest2008', i)][-1]
s

'AMBest2008b'

time: 15 ms (started: 2025-05-28 15:07:13 +01:00)


In [71]:
chr(ord('a') + 1)

'b'

time: 0 ns (started: 2025-05-28 15:07:59 +01:00)


In [72]:
t = s[:-1] + chr(ord(s[-1])+1)
t

'AMBest2008c'

time: 0 ns (started: 2025-05-28 15:08:14 +01:00)


In [137]:
x = ''
y = x or 'aaasd'
y

'aaasd'

time: 0 ns (started: 2025-05-28 23:13:09 +01:00)


In [126]:
(Path().home() / 'downloads').exists()

True

time: 0 ns (started: 2025-05-28 15:43:57 +01:00)


In [131]:
import archivum.cli as cli

time: 0 ns (started: 2025-05-28 17:49:10 +01:00)


In [136]:
cli.start_uber_repl('', False)

RuntimeError: asyncio.run() cannot be called from a running event loop

time: 485 ms (started: 2025-05-28 17:53:31 +01:00)


In [134]:
if '': 10

time: 0 ns (started: 2025-05-28 17:50:57 +01:00)


In [138]:
from archivum import DEFAULT_CONFIG_FILE, BASE_DIR
DEFAULT_CONFIG_FILE, BASE_DIR

('uber-library', WindowsPath('C:/Users/steve/AppData/Local/archivum'))

time: 16 ms (started: 2025-05-28 23:15:24 +01:00)


In [140]:
l = list('abcd')
l.remove('d')
l

['a', 'b', 'c']

time: 0 ns (started: 2025-05-29 08:08:42 +01:00)


In [141]:
l.remove('d')

ValueError: list.remove(x): x not in list

time: 16 ms (started: 2025-05-29 08:08:49 +01:00)


In [149]:
c = type('EmptyLibrary', (), {'name': 'No library open'})

time: 0 ns (started: 2025-05-29 09:25:56 +01:00)


In [150]:
l = c()

time: 0 ns (started: 2025-05-29 09:25:56 +01:00)


In [151]:
l.name

'No library open'

time: 0 ns (started: 2025-05-29 09:25:57 +01:00)


In [152]:
len('archivum.cli.list_libraries')

27

time: 0 ns (started: 2025-05-29 09:53:44 +01:00)


In [162]:
a = 'a long string of text askdfj a'
print(len(a))
f'{a[-120:].strip():<20}'

30


'a long string of text askdfj a'

time: 0 ns (started: 2025-05-29 10:21:04 +01:00)


In [175]:
df = arcl.Library.list_deets()

10:40:06 | archivum.library.__init__      | INFO     | config_path = C:\Users\steve\AppData\Local\archivum\books.archivum-config
10:40:06 | archivum.library.__init__      | INFO     | config_path = C:\Users\steve\AppData\Local\archivum\uber-library.archivum-config
10:40:06 | archivum.library.list_deets    | INFO     | key            name                             description  \
value         books                        downloaded books   
value  Uber Library  Mendeley port from May 2025 plus books   

key                         bibtex_file     pdf_dir        text_dir_name  \
value    \S\Telos\biblio\books-test.bib    \S\BOOKS  \temp\pdf-full-text   
value  \S\Telos\biblio\uber-library.bib  \S\Library  \temp\pdf-full-text   

key    extractor                watched_dirs  
value  pdftotext  [c:\users\steve\downloads]  
value  pdftotext  [c:\users\steve\downloads]  
time: 62 ms (started: 2025-05-29 10:40:06 +01:00)


In [176]:
fGT(df)

name,description,bibtex_file,pdf_dir,text_dir_name,extractor,watched_dirs
books,downloaded books,\S\Telos\biblio\books-test.bib,\S\BOOKS,\temp\pdf-full-text,pdftotext,['c:\\users\\steve\\downloads']
Uber Library,Mendeley port from May 2025 plus books,\S\Telos\biblio\uber-library.bib,\S\Library,\temp\pdf-full-text,pdftotext,['c:\\users\\steve\\downloads']


time: 31 ms (started: 2025-05-29 10:40:11 +01:00)


In [169]:
arcl.Library.get_library_list()

[WindowsPath('C:/Users/steve/AppData/Local/archivum/books.archivum-config'),
 WindowsPath('C:/Users/steve/AppData/Local/archivum/uber-library.archivum-config')]

time: 16 ms (started: 2025-05-29 10:36:49 +01:00)


In [171]:
print(arcl.Library.list())

books.archivum-config
uber-library.archivum-config
time: 16 ms (started: 2025-05-29 10:36:57 +01:00)


In [177]:
lib

Library(uber-library.archivum-config)

time: 16 ms (started: 2025-05-29 10:42:07 +01:00)


In [178]:
lib.stats()

,references,documents
objects,6118,6132
no children,167,156
children,5951,5976
1 child,5833,5883
2 children,108,86
3 children,10,4
4+ children,0,3


time: 31 ms (started: 2025-05-29 10:42:11 +01:00)


In [181]:
lib.distinct_values_by_field()

,distinct,missing
tag,6118,0
type,10,0
author,4662,5
doi,2539,3431
journal,1225,2643
pages,2749,2233
title,5902,0
volume,454,3143
year,100,256
publisher,169,5579


time: 31 ms (started: 2025-05-29 10:42:22 +01:00)


In [182]:
lib.distinct('type')

['Report',
 'article',
 'book',
 'incollection',
 'inproceedings',
 'manual',
 'misc',
 'phdthesis',
 'techreport',
 'unpublished']

time: 16 ms (started: 2025-05-29 10:42:35 +01:00)


In [186]:
lib.ref_df['type'].value_counts().to_frame('count').sort_values('count', ascending=False).head(20)

,count
type,
article,4980
book,418
techreport,413
misc,133
incollection,59
unpublished,59
inproceedings,44
phdthesis,10
Report,1


time: 0 ns (started: 2025-05-29 10:44:09 +01:00)


In [198]:
lib.distinct_value_counts('type')

,count
type,
article,10561
book,697
techreport,585
unpublished,313
misc,200
incollection,106
inproceedings,79
phdthesis,10
Report,2


time: 0 ns (started: 2025-05-29 10:51:21 +01:00)


In [196]:
pd.DataFrame([]).empty

True

time: 0 ns (started: 2025-05-29 10:50:23 +01:00)


In [197]:
str(fGT(None))

''

time: 0 ns (started: 2025-05-29 10:50:49 +01:00)


In [201]:
lib.list_deets()#.reset_index(drop=False)

10:55:10 | archivum.library.__init__      | INFO     | config_path = C:\Users\steve\AppData\Local\archivum\books.archivum-config
10:55:10 | archivum.library.__init__      | INFO     | config_path = C:\Users\steve\AppData\Local\archivum\uber-library.archivum-config
10:55:10 | archivum.library.list_deets    | INFO     | key  index          name                             description  \
0    value         books                        downloaded books   
1    value  Uber Library  Mendeley port from May 2025 plus books   

key                       bibtex_file     pdf_dir        text_dir_name  \
0      \S\Telos\biblio\books-test.bib    \S\BOOKS  \temp\pdf-full-text   
1    \S\Telos\biblio\uber-library.bib  \S\Library  \temp\pdf-full-text   

key  extractor                watched_dirs  
0    pdftotext  [c:\users\steve\downloads]  
1    pdftotext  [c:\users\steve\downloads]  


key,index,name,description,bibtex_file,pdf_dir,text_dir_name,extractor,watched_dirs
0,value,books,downloaded books,\S\Telos\biblio\books-test.bib,\S\BOOKS,\temp\pdf-full-text,pdftotext,[c:\users\steve\downloads]
1,value,Uber Library,Mendeley port from May 2025 plus books,\S\Telos\biblio\uber-library.bib,\S\Library,\temp\pdf-full-text,pdftotext,[c:\users\steve\downloads]


time: 62 ms (started: 2025-05-29 10:55:10 +01:00)


In [204]:
fGT(lib.stats().reset_index(drop=False))

index,references,documents
objects,"6,118","6,132"
no children,167,156
children,"5,951","5,976"
1 child,"5,833","5,883"
2 children,108,86
3 children,10,4
4+ children,0,3


time: 31 ms (started: 2025-05-29 10:56:58 +01:00)


In [3]:
from archivum.parser import ArcLexer, ArcParser , grammar

In [6]:
grammar()

C:\Users\steve\Documents\CloudStation\TELOS\Python\archivum_project\temp\grammar.md


'GRAMMAR\n=======\n\nclause_list         \t::= \n                    \t | clause_list clause\n                    \t | clause\n\nclause              \t::= TOP NUMBER\n                    \t | flags\n                    \t | regexes\n                    \t | SELECT select_list\n                    \t | WHERE where_clause_expression\n                    \t | ORDER_BY column_sort_list\n\nflags               \t::= FLAG flags\n                    \t | FLAG\n\nregexes             \t::= regexes AND regex\n                    \t | regex\n\nregex               \t::= IDENTIFIER TILDE REGEX_SLASHED\n                    \t | IDENTIFIER TILDE IDENTIFIER\n                    \t | BANG REGEX_SLASHED\n                    \t | BANG IDENTIFIER\n\nselect_list         \t::= select_list "," select_item\n                    \t | select_item\n\nselect_item         \t::= IDENTIFIER\n                    \t | NOT IDENTIFIER\n                    \t | STAR\n\nwhere_clause_expression\t::= where_clause_expression A

In [1]:
10

10

In [2]:
import archivum.lazy_parser as arcz

In [20]:
import archivum.parser as arcp
import archivum.library as arcl
from archivum.utilities import fGT

In [33]:
lib = arcl.Library('uber-library')

12:02:29 | archivum.library.__init__      | INFO     | config_path = C:\Users\steve\AppData\Local\archivum\uber-library.archivum-config


In [1]:
querex_test_cases = [
    '',
    'top 4',
    'recent',
    'recent top 3',
    'verbose recent top 17',
    'select *',
    'top 10 select *',
    'where year == 2024',
    'where year == "2024"',
    'where type == "book"',
    '! Delbaen',
    '! /Wang, R/ and journal ~ Annals',
    'recent top 3 author ~ /Wang, R/',
    'verbose top 5 recent select journal author ~ /Wang, R/',
    'top 5 select journal author ~ /Wang, R/',
    'top 6 order author',
    'top 7 order journal',
    'top 8 select journal where year == 2024 order author',
    'verbose top 9 select journal where year == 2024 order -journal, author',
    'recent top 10  verbose select *, -c ! /Wang, R/ and journal ~ [A-J]+ where year == 2024 and publisher ==  "Springer" and mod > 2024-05',
    'top 10 select c, d, -a, e recent ! /Wang, R/ and journal ~ [A-J]+ where year == 2024 and publisher ==  "Springer" and mod > 2024 order name, year', 
    'error',
]

In [ ]:
import archivum.parser as arcp
import archivum.library as arcl
from archivum.utilities import fGT
lib = arcl.Library('uber-library')

In [44]:
for q in querex_test_cases[-1:]:
    print(repr(q))
    try:
        r = lib.ref_df.querex(q) 
    except ValueError as e:
        print('Synt error', e)
    else:
        # print(sorted(r.columns))
        display(fGT(r.head(10)))

'error'
Parsing Error: Unexpected end of input
Synt error Parsing Error: Unexpected end of input


In [172]:
%load_ext autotime

time: 0 ns (started: 2025-05-29 23:07:48 +01:00)


In [ ]:
arck.parse_test('recent top 10  verbose select a, b, -c, d name ~ Wang and xx ~ yy AND journal ~ [A-E]+', 1, 1);

In [ ]:
arck.parse_test('recent top 10  verbose select *, -c ! /Wang, R/ and journal ~ [A-J]+ where year == 2024 and publisher ==  "Springer" and mod > 2024-05', 1, 1);

In [ ]:
arck.parse_test('top 10 select c, d, -a, e recent ! /Wang, R/ and journal ~ [A-J]+ where year == 2024 and publisher ==  "Springer" and mod > 2024 order name, year', 0, 1);

In [ ]:
arck.parse_test(12, '''
select g, h, i 
verbose 
select k 
order f recent 
order name, year, -c, d, -e
''', 0, 1);

In [11]:
%load_ext autotime

time: 0 ns (started: 2025-05-29 23:51:04 +01:00)


In [14]:
isinstance(12, int)

True

time: 0 ns (started: 2025-05-29 23:56:11 +01:00)


In [3]:
import archivum.parser as arck  
from pprint import pprint as pp

In [15]:
ans = {}
for i, q in enumerate(querex_test_cases):
    # print('\n' + '-' * 80)
    # print('query: ' + repr(q))
    try:
        # r = arck.parse_test(i, q, 0, 0)
        r = arck.parser(q, False)
    except Exception as e:
        print('*' * 20 + 'ERROR' + '*' * 20)
        print(f'Error [#{i}] {e}')
        ans[i] = (q, 'ERROR')
    else:
        ans[i] = (q, r)
        pass
        # duplicates output 
        # print(r)

********************ERROR********************
Error [#21] Parsing Error: Unexpected end-of-input. Expected one of: 
	* TILDE
	* TILDE

time: 687 ms (started: 2025-05-29 23:56:19 +01:00)


In [ ]:
ans

In [163]:
from greater_tables import GT

time: 0 ns (started: 2025-05-30 15:32:50 +01:00)


In [150]:
df = lib.ref_df.querex('top 10 recent')

time: 31 ms (started: 2025-05-30 15:24:09 +01:00)


In [180]:
f = GT(df, year_cols=['year'], raw_cols=['index'])
f

index,tag,type,year,author,title
1642,Nemirovski209YYYY,article,209,"Nemirovski, Arkadi S. and Todd, Michael J.",Interior-point methods for optimization
2123,Reed2025,misc,2025,"Reed, Lord and Hodge, Lord and President, Deputy and Clayton, Doyle",JUDGMENT For Women Scotland Ltd (Appellant) v The Scottish Ministers (Respondent) before Lady Rose Lady Simler
3254,Centrone2025,article,2025,"Centrone, Francesca and {Rosazza Gianin}, Emanuela",Capital Allocation Rules and Generalized Collapse to the Mean
1678,Dhaene2025,article,2025,"Dhaene, Jan and Robert, Christian Y. and Cheung, Ka Chun and Denuit, Michel M.",An axiomatic characterization of the quantile risk-sharing rule
6042,Vecchi2025,article,2025,"Vecchi, Corrado De and Scherer, Matthias",Pricing insurance contracts with an existing portfolio as background risk
5502,Chen2025a,unpublished,2025,"Chen, Yuyu and Hu, Taizhong and Wang, Ruodu and Zou, Zhenfeng",Diversification for infinite-mean Pareto models without risk aversion
4032,Aase2025,unpublished,2025,"Aase, Knut K.",Pareto Optimal Insurance Policies: Kinks with or without frictions
1357,BarLev2025,article,2025,"Bar-Lev, Shaul K. and Letac, Gérard",Chainability of infinitely divisible measures
1260,Cheung2025,article,2025,"Cheung, Ka Chun and Yam, Sheung Chi Phillip and Yuen, Fei Lung and Zhang, Yiying",Optimal design of reinsurance contracts under adverse selection with a continuum of types
2528,Vytnova2025,book,2025,"Vytnova, Polina L. and Wormell, Caroline L.",Hausdorff dimension of the Apollonian gasket


time: 32 ms (started: 2025-05-30 15:51:48 +01:00)


In [181]:
f.break_penalties

[10, 1, 1, 1, 1, 1]

time: 0 ns (started: 2025-05-30 15:52:03 +01:00)


In [165]:
len(df.columns), len(f.df.columns)

(5, 6)

time: 0 ns (started: 2025-05-30 15:33:42 +01:00)


In [156]:
from archivum.utilities import info

time: 0 ns (started: 2025-05-30 15:24:51 +01:00)


In [160]:
f.df_formatters

[<function greater_tables.greater_tables.GT.__init__.<locals>.wrapped_default_formatter(x)>,
 <function greater_tables.greater_tables.GT.__init__.<locals>.wrapped_default_formatter(x)>,
 <bound method GT.default_raw_formatter of GreaterTable(df_id=TQMULYJKHVQ2P)>,
 <function greater_tables.greater_tables.GT.__init__.<locals>.wrapped_default_formatter(x)>,
 <function greater_tables.greater_tables.GT.__init__.<locals>.wrapped_default_formatter(x)>]

time: 0 ns (started: 2025-05-30 15:25:30 +01:00)


In [161]:
f.dt

tag       object
type      object
year      object
author    object
title     object
dtype: object

time: 16 ms (started: 2025-05-30 15:26:05 +01:00)


In [195]:
info(f)

id,<class 'greater_tables.greater_tables.GT'>
break_penalties,"[10, 1, 1, 1, 1, 1]"
caption,"Query: top 10 recent, 6118, showing top 10 of ..."
caption_align,center
column_change_level,0 0 1 0 2 0 3 0 4 0 dtype: int64
date_col_indices,[]
debug,False
default_date_str,%Y-%m-%d
default_float_formatter,False
default_float_str,"{x:,.3f}"
default_formatters,{}


time: 32 ms (started: 2025-05-30 16:00:59 +01:00)


In [196]:
self = f

time: 0 ns (started: 2025-05-30 16:01:37 +01:00)


In [197]:
dict(zip(range(len(f.df.columns)), [True if i < 5 else False for i in self.break_penalties]))

{0: False, 1: True, 2: True, 3: True, 4: True, 5: True}

time: 0 ns (started: 2025-05-30 16:01:54 +01:00)


In [ ]:
for q in querex_test_cases:
    print(repr(q))
    r = arcp.parse_test(q, 0, 0)
    pp(r)

In [199]:
f.df

,index,tag,type,year,author,title
0,1642,Nemirovski209YYYY,article,209,"Nemirovski, Arkadi S. and Todd, Michael J.",Interior-point methods for optimization
1,2123,Reed2025,misc,2025,"Reed, Lord and Hodge, Lord and President, Depu...",JUDGMENT For Women Scotland Ltd (Appellant) v ...
2,3254,Centrone2025,article,2025,"Centrone, Francesca and {Rosazza Gianin}, Eman...",Capital Allocation Rules and Generalized Colla...
3,1678,Dhaene2025,article,2025,"Dhaene, Jan and Robert, Christian Y. and Cheun...",An axiomatic characterization of the quantile ...
4,6042,Vecchi2025,article,2025,"Vecchi, Corrado De and Scherer, Matthias",Pricing insurance contracts with an existing p...
5,5502,Chen2025a,unpublished,2025,"Chen, Yuyu and Hu, Taizhong and Wang, Ruodu an...",Diversification for infinite-mean Pareto model...
6,4032,Aase2025,unpublished,2025,"Aase, Knut K.",Pareto Optimal Insurance Policies: Kinks with ...
7,1357,BarLev2025,article,2025,"Bar-Lev, Shaul K. and Letac, Gérard",Chainability of infinitely divisible measures
8,1260,Cheung2025,article,2025,"Cheung, Ka Chun and Yam, Sheung Chi Phillip an...",Optimal design of reinsurance contracts under ...
9,2528,Vytnova2025,book,2025,"Vytnova, Polina L. and Wormell, Caroline L.",Hausdorff dimension of the Apollonian gasket


time: 16 ms (started: 2025-05-30 16:02:40 +01:00)


In [194]:
f.df.author.str.split(pat='[^\w]', regex=True, expand=True).fillna('').map(len).max(axis=1).max()

10

time: 0 ns (started: 2025-05-30 15:59:46 +01:00)


In [188]:
df.dtypes

tag       object
type      object
year      object
author    object
title     object
dtype: object

time: 0 ns (started: 2025-05-30 15:56:42 +01:00)


In [187]:
np.all([i==object for i in df.dtypes.values])

True

time: 0 ns (started: 2025-05-30 15:56:33 +01:00)


In [183]:
f.df.map()

,index,tag,type,year,author,title
0,1642,Nemirovski209YYYY,article,209,"Nemirovski, Arkadi S. and Todd, Michael J.",Interior-point methods for optimization
1,2123,Reed2025,misc,2025,"Reed, Lord and Hodge, Lord and President, Depu...",JUDGMENT For Women Scotland Ltd (Appellant) v ...
2,3254,Centrone2025,article,2025,"Centrone, Francesca and {Rosazza Gianin}, Eman...",Capital Allocation Rules and Generalized Colla...
3,1678,Dhaene2025,article,2025,"Dhaene, Jan and Robert, Christian Y. and Cheun...",An axiomatic characterization of the quantile ...
4,6042,Vecchi2025,article,2025,"Vecchi, Corrado De and Scherer, Matthias",Pricing insurance contracts with an existing p...
5,5502,Chen2025a,unpublished,2025,"Chen, Yuyu and Hu, Taizhong and Wang, Ruodu an...",Diversification for infinite-mean Pareto model...
6,4032,Aase2025,unpublished,2025,"Aase, Knut K.",Pareto Optimal Insurance Policies: Kinks with ...
7,1357,BarLev2025,article,2025,"Bar-Lev, Shaul K. and Letac, Gérard",Chainability of infinitely divisible measures
8,1260,Cheung2025,article,2025,"Cheung, Ka Chun and Yam, Sheung Chi Phillip an...",Optimal design of reinsurance contracts under ...
9,2528,Vytnova2025,book,2025,"Vytnova, Polina L. and Wormell, Caroline L.",Hausdorff dimension of the Apollonian gasket


time: 15 ms (started: 2025-05-30 15:52:56 +01:00)


In [185]:
list(f.df.applymap(len).max(axis=0).items())

[('index', 4),
 ('tag', 17),
 ('type', 11),
 ('year', 4),
 ('author', 80),
 ('title', 110)]

time: 16 ms (started: 2025-05-30 15:53:29 +01:00)


In [201]:
import table_w as tw

time: 16 ms (started: 2025-05-30 16:06:55 +01:00)


In [226]:
tw.optimize_column_widths(self, 120)

        breakable_cols  min_possible_widths  ideal_widths  col_widths
tag               True                   17            17       17.00
type              True                   11            11       11.00
year              True                    3             5        3.76
author            True                   10            80       36.57
title             True                   16           110       51.67
min_possible_widths     57.0
ideal_widths           223.0
col_widths             120.0
dtype: float64


{'tag': 17, 'type': 11, 'year': 3.76, 'author': 36.57, 'title': 51.67}

time: 16 ms (started: 2025-05-30 16:21:28 +01:00)


In [234]:
f = GT(df, show_index=False, year_cols=['year'], max_table_width=90)

time: 0 ns (started: 2025-05-30 16:33:23 +01:00)


In [264]:
len(' ignition   │ shoes       │ infectious   │   Chairperson Inform Favorite year │  Portals Basil Burdensome date  │  Prospects Kenya Harmony date  │   Torque Unleashed Prophecy float │ Tripartite Problems Gratuitously str   ')

222

time: 16 ms (started: 2025-05-30 16:52:58 +01:00)


In [239]:
from greater_tables import TestDFGenerator

time: 0 ns (started: 2025-05-30 16:37:54 +01:00)


In [ ]:
print(s.)

│ flora   │ sticker   │ identities   │   AAA   │    BBB │       CCC │ DDD      │     EEE │   FFF   │     GGG │     HHH │     III │     JJJ │


(array([10, 12, 15, 10,  9, 12, 11, 10, 10, 10, 10, 10, 10]), 139)

time: 0 ns (started: 2025-05-30 16:59:39 +01:00)


In [431]:
len('citizenship')

11

time: 0 ns (started: 2025-05-30 21:43:57 +01:00)


In [ ]:
tdf = TestDFGenerator()
ts = tdf.test_suite()
t

In [437]:
80 + 9*4, 80 - v.shape[1] * 4

(116, 48)

time: 0 ns (started: 2025-05-30 21:51:54 +01:00)


# COLUMN WIDTHS

In [ ]:
for k, v in ts.items():
    # if k != 'multiindex': continue
    cw_target = 80
    input_target = cw_target - 4 * v.shape[1]
    print(f'{cw_target = } and {input_target = }')
    f = (GT(v.head(), show_index=True, max_table_width=80 - 4 * v.shape[1], debug=True))
    analysis(f)
    # break
    print('=*'*80)

In [464]:
def analysis(f):
    """look at the text repr"""
    s = str(f)
    ans = list(map(len, s.split('\n')))
    if min(ans) != max(ans): print('Weird, cols not all the same size', min(ans), max(ans))
    print('Actual text table width by row  ', min(ans))
    ans = []
    s1 = s.split('\n')[1]
    print('top row                         ', s1)
    for i, c in enumerate(s1):
        # print(i,c)
        if c == '│':
            ans.append(i)
    ans = np.diff(np.array(ans))
    print('actual widths by column         ', ans)
    print('acutal total width              ', ans.sum() + 1)
    dcw = getattr(f, '_debug_col_widths', None)
    if dcw is not None:
        ans = np.array(ans)
        ans = ans - 4
        dcw['actual'] = list(ans) + [ans.sum() + 1]
        try:
            display(dcw.astype(int))
        except:
            display(dcw)
    print('formatted output')
    print(s)

time: 0 ns (started: 2025-05-30 22:05:33 +01:00)


In [465]:
for k, v in ts.items():
    # if k != 'multiindex': continue
    cw_target = 80
    input_target = cw_target - 4 * v.shape[1]
    print(f'{cw_target = } and {input_target = }')
    f = (GT(v.head(), show_index=True, max_table_width=80 - 4 * v.shape[1], debug=True))
    analysis(f)
    # break
    print('=*'*80)

cw_target = 80 and input_target = 48
Actual text table width by row   123
top row                          │ vestige   │ bluffing   │  citizenship  │ demographic   │  difficultly  │   glaze │   memorandum │   positives │ truce   │
actual widths by column          [12 13 16 16 16 10 15 14 10]
acutal total width               123


,break_penalties,breakable_cols,min_possible_widths,ideal_widths,col_widths,headers_cw,headers_ideal,headers_min,actual
Column,,,,,,,,,
vestige,10,0,6,6,4,4,7,7,8
bluffing str,1,1,12,12,7,5,12,8,9
citizenship datetime,5,0,10,10,6,7,20,11,12
demographic str,1,1,10,10,6,7,15,11,12
difficultly datetime,5,0,10,10,6,7,20,11,12
glaze float,10,0,7,7,4,3,11,5,6
memorandum float,10,0,9,9,5,6,16,10,11
positives int,10,0,9,9,5,6,13,9,10
truce str,1,1,9,9,5,3,9,5,6


formatted output
┍━━━━━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━━┑
│ vestige   │ bluffing   │  citizenship  │ demographic   │  difficultly  │   glaze │   memorandum │   positives │ truce   │
│           │ str        │   datetime    │ str           │   datetime    │   float │        float │         int │ str     │
┝━━━━━━━━━━━┿━━━━━━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━━━┿━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┿━━━━━━━━━┥
│ 16,553    │ brunch     │  2020-08-02   │ deters        │  2009-02-27   │ 0.59401 │     246.830M │    937.727M │ clipb   │
│           │            │               │               │               │         │              │             │ oard    │
├───────────┼────────────┼───────────────┼───────────────┼───────────────┼─────────┼──────────────┼─────────────┼─────────┤
│ 30,023    │ arbor      │  2024-12-18   │ slice         │  2009-02-27   │   6e-05 │       8.244M │         NaN │ b

,break_penalties,breakable_cols,min_possible_widths,ideal_widths,col_widths,headers_cw,headers_ideal,headers_min,actual
Column,,,,,,,,,
comer,5,0,10,10,10,5,5,5,9
Billionaire Convicted Erected float,10,0,9,9,9,21,35,11,12
Brainless Unleashed Insignificant str,1,1,13,13,13,23,37,13,14
Opportune Rambling Impedes float,10,0,7,7,7,19,32,9,10
total,26,1,39,39,39,68,109,38,46


formatted output
┍━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ comer      │   Billionaire │ Brainless       │   Opportune │
│            │     Convicted │ Unleashed       │    Rambling │
│            │       Erected │ Insignificant   │     Impedes │
│            │         float │ str             │       float │
┝━━━━━━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ 2006-05-29 │        4.978M │ fuses           │           0 │
├────────────┼───────────────┼─────────────────┼─────────────┤
│ 2006-10-01 │      163.377k │ nan             │     0.71859 │
├────────────┼───────────────┼─────────────────┼─────────────┤
│ 2006-10-31 │      198.361k │ degeneration    │     0.00013 │
├────────────┼───────────────┼─────────────────┼─────────────┤
│ 2007-01-16 │        2.846G │ doomsday        │     0.03598 │
├────────────┼───────────────┼─────────────────┼─────────────┤
│ 2007-04-20 │       423.417 │ consolidating   │       1e-05 │
┕━━━━━━━━━━━━┷━━━━━━━━━━━━━━━┷━━━━━━━━

...

formatted output
┍━━━━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━━━┯━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┑
│ configure   │ smarts      │ tandem   │   Escort │   Fortified │   Reconfigure │ Schoolchildren   │    Surplus │
│             │             │          │   Unlike │      Basket │   Sharif Self │ Pain Hovering    │   Dearborn │
│             │             │          │    Hairy │    Unawares │         float │ str              │   Illinois │
│             │             │          │    float │         int │               │                  │      float │
┝━━━━━━━━━━━━━┿━━━━━━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━━┿━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━┥
│ 22,120      │ ascertainin │ 41,099   │  -46.593 │    8,919.00 │             0 │ piggy            │    0.10071 │
│             │ g           │          │        k │             │               │                  │            │
├─────────────┼─────────────┼──────────┼──────────┼─────────────┼──────

break_penalties breakable_cols  \
                                                                
                   dixie                  10.0          False   
newer   bless      austral                 5.0          False   
        watertight unhappy                10.0          False   
updated bless      clerical               10.0          False   
        redefined  emerging               10.0          False   
        watertight swallowing             10.0          False   
total                                      NaN            NaN   

                               min_possible_widths  ideal_widths  col_widths  \
                                                                               
                   dixie                       6.0           6.0         6.0   
newer   bless      austral                    10.0          10.0        10.0   
        watertight unhappy                     8.0           8.0         8.0   
updated bless      clerical                    9.0           9.0         9.0   
        redefined  emerging                    9.0           9.0         9.0   
        watertight swallowing                  5.0           5.0         5.0   
total                                          NaN           NaN         NaN   

                               headers_cw  headers_ideal  headers_min  actual  
                                                                               
                   dixie              3.0            3.0          1.0       7  
newer   bless      austral            3.0            3.0          1.0      11  
        watertight unhappy            3.0            3.0          1.0       9  
updated bless      clerical           3.0            3.0          1.0      10  
        redefined  emerging           3.0            3.0          1.0      10  
        watertight swallowing         3.0            3.0          1.0       6  
total                                 NaN            NaN          NaN      54

formatted output
┍━━━━━━━━━━┯━━━━━━━━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━━┑
│ ('',     │  ('newer',   │   ('newer' │   ('updated │   ('updated │   ('upd │
│ '', 'd   │   'bless',   │   , 'water │          ', │   ', 'redef │   ated' │
│ ixie')   │  'austral')  │   tight',  │   'bless',  │   ined', 'e │   , 'wa │
│          │              │   'unhappy │   'clerical │   merging') │   terti │
│          │              │         ') │          ') │             │   ght', │
│          │              │            │             │             │   'swal │
│          │              │            │             │             │   lowin │
│          │              │            │             │             │     g') │
┝━━━━━━━━━━┿━━━━━━━━━━━━━━┿━━━━━━━━━━━━┿━━━━━━━━━━━━━┿━━━━━━━━━━━━━┿━━━━━━━━━┥
│ 12,901   │  2032-11-18  │   1,992.00 │       7.064 │     281.101 │   1,994 │
├──────────┼──────────────┼────────────┼─────────────┼─────────────┼─────────┤
│ 16,581   │  2029-08-28  │   1,991

break_penalties breakable_cols  \
                                                                        
                           shamrock               10.0          False   
                           auspicious              1.0           True   
                           viper                  10.0          False   
commercially buttered      brodie                 10.0          False   
                           cursor                 10.0          False   
                           grading                10.0          False   
             fertilization attainable              5.0          False   
                           sympathize             10.0          False   
             foils         enumerate              10.0          False   
                           pointless              10.0          False   
                           promotes               10.0          False   
textile      buttered      infuriates             10.0          False   
             foils         accounting              5.0          False   
total                                              NaN            NaN   

                                       min_possible_widths  ideal_widths  \
                                                                           
                           shamrock                    6.0           6.0   
                           auspicious                  8.0           8.0   
                           viper                       6.0           6.0   
commercially buttered      brodie                      9.0           9.0   
                           cursor                      6.0           6.0   
                           grading                     9.0           9.0   
             fertilization attainable                 10.0          10.0   
                           sympathize                  5.0           5.0   
             foils         enumerate                   9.0           9.0   
                           pointless                  11.0          11.0   
                           promotes                    8.0           8.0   
textile      buttered      infuriates                  9.0           9.0   
             foils         accounting                 10.0          10.0   
total                                                  NaN           NaN   

                                       col_widths  headers_cw  headers_ideal  \
                                                                               
                           shamrock           2.0         3.0            3.0   
                           auspicious         3.0         3.0            3.0   
                           viper              2.0         3.0            3.0   
commercially buttered      brodie             3.0         3.0            3.0   
                           cursor             2.0         3.0            3.0   
                           grading            3.0         3.0            3.0   
             fertilization attainable         4.0         3.0            3.0   
                           sympathize         2.0         3.0            3.0   
             foils         enumerate          3.0         3.0            3.0   
                           pointless          4.0         3.0            3.0   
                           promotes           3.0         3.0            3.0   
textile      buttered      infuriates         3.0         3.0            3.0   
             foils         accounting         4.0         3.0            3.0   
total                                         NaN         NaN            NaN   

                                       headers_min  actual  
                                                            
                           shamrock            1.0       3  
                           auspicious          1.0       4  
                           viper               1.0       3  
commercially buttered      brodie              1.0       6  
      

formatted output
┍━━━━━━┯━━━━━━━┯━━━━━━┯━━━━━━━━━┯━━━━━━┯━━━━━━━┯━━━━━━━━┯━━━━━━━┯━━━━━━━┯━━━━━━━━┯━━━━━━━━━┯━━━━━━━┯━━━━━━━━┑
│ ('   │ (''   │ ('   │     ('c │   (' │   ('c │  ('co  │    (' │   ('c │   ('co │     ('c │   ('t │  ('te  │
│ ',   │ ,     │ ',   │     omm │   co │   omm │  mmer  │    co │   omm │   mmer │     omm │   ext │  xtil  │
│ ''   │ '',   │ ''   │     erc │   mm │   erc │  cial  │    mm │   erc │   cial │     erc │   ile │  e',   │
│ ,    │ 'au   │ ,    │     ial │   er │   ial │  ly',  │    er │   ial │   ly', │     ial │   ',  │  'foi  │
│ 's   │ spi   │ 'v   │     ly' │   ci │   ly' │  'fer  │    ci │   ly' │   'foi │     ly' │   'bu │  ls',  │
│ ha   │ cio   │ ip   │     , ' │   al │   , ' │  tili  │    al │   , ' │   ls', │     , ' │   tte │  'acc  │
│ mr   │ us'   │ er   │     but │   ly │   but │  zati  │    ly │   foi │   'poi │     foi │   red │  ount  │
│ oc   │ )     │ ')   │     ter │   ', │   ter │  on',  │    ', │   ls' │   ntle │     ls' │   ',  │  i